In [5]:
import os
import pandas as pd
import numpy as np

import locale
from locale import atof
import xlsxwriter

from PreProcessingUtil import preprocessing

In [ ]:
!pip install fastdtw

In [ ]:
import numpy as np
from scipy.spatial.distance import euclidean

from fastdtw import fastdtw

In [7]:
df = pd.read_excel(
     os.path.join("../datasets", "training", "PM10.xlsx"),
     engine='openpyxl',
     parse_dates=True,
     thousands='.'
)
print (df)

       Unnamed: 0    Aksaray   Besiktas   Buyukada  Catladıkapı    Esenler  \
0               0  42.200001  37.599998   9.600000    24.100000  28.100000   
1               1  41.799999  42.000000  12.000000    27.299999  34.200001   
2               2  44.200001  39.000000  12.800000    40.200001  33.099998   
3               3  41.700001  37.599998  13.800000    41.599998  33.799999   
4               4  49.700001  48.500000  16.400000    40.900002  36.000000   
...           ...        ...        ...        ...          ...        ...   
19995       19995  23.600000  11.100000  13.000000    13.800000  12.200000   
19996       19996  25.900000  17.400000   8.700000    17.200001  18.200001   
19997       19997  27.600000  30.000000  13.300000    14.600000  18.299999   
19998       19998  19.700001  18.600000  30.600000    14.800000  22.000000   
19999       19999  27.100000  13.200000  26.200001    16.400000  31.299999   

         Kadıkoy  Kandilli     Kartal  Mecidiyekoy   Umraniye  

In [10]:
import numpy as np
from scipy.spatial.distance import euclidean

from fastdtw import fastdtw

x = np.array([[1,1], [2,2], [3,3], [4,4], [5,5]])
y = np.array([[2,2], [3,3], [4,4]])
distance, path = fastdtw(x, y, dist=euclidean)
print(distance)

2.8284271247461903


In [16]:
sehirList = ['Aksaray', 'Besiktas','Buyukada','Catladıkapı','Esenler','Kadıkoy','Kandilli','Kartal','Mecidiyekoy','Umraniye']
n_sensors = len(sehirList)

In [18]:

# Preparing adjacency matrix for the sensor nodes
dtw_matrix = np.zeros ((n_sensors, n_sensors))

ix = 0
for s_sehir in sehirList:
    o_ix = 0
    for o_sehir in sehirList:
        distance, path = fastdtw(df[s_sehir], df[o_sehir], dist=euclidean)
        dtw_matrix [ix][o_ix] = distance
        o_ix += 1
    ix += 1


In [22]:
# Take the absolute of the difference between all values and the max value
# Then divide it by the max value to get the weighted adjacency matrix
# Add 1 to the max value so that the weights will never be lost
max_val = np.max (dtw_matrix) + 1
dtw_matrix = (max_val - dtw_matrix)/max_val
dtw_matrix

array([[1.00000000e+00, 4.45086783e-01, 4.53423780e-01, 5.32355890e-01,
        6.30163232e-01, 6.35399538e-01, 3.75464704e-01, 4.98378437e-01,
        4.64685538e-01, 5.39603568e-01],
       [5.30907880e-01, 1.00000000e+00, 6.00842666e-01, 6.94119237e-01,
        6.31160736e-01, 6.27805640e-01, 5.74407828e-01, 3.73059408e-01,
        3.57943383e-01, 6.95721102e-01],
       [4.74394411e-01, 6.28788178e-01, 1.00000000e+00, 7.08955514e-01,
        5.43727484e-01, 5.38513468e-01, 7.68863204e-01, 3.81356032e-01,
        1.86586592e-01, 5.99261377e-01],
       [5.27453799e-01, 7.14560642e-01, 6.83591132e-01, 1.00000000e+00,
        6.17029679e-01, 5.70370428e-01, 6.90865153e-01, 4.00860572e-01,
        3.88520264e-01, 6.15270995e-01],
       [6.35289484e-01, 6.07648881e-01, 5.22595825e-01, 6.23576659e-01,
        1.00000000e+00, 6.87408423e-01, 4.47231593e-01, 4.47393246e-01,
        3.58353327e-01, 6.26396916e-01],
       [6.39764296e-01, 6.23940337e-01, 5.38041763e-01, 5.72628722e-01,
   

In [23]:
# export to excel
workbook = xlsxwriter.Workbook('../datasets/training/dtw_adj.xlsx')
worksheet = workbook.add_worksheet()

row = 0

for col, data in enumerate(dtw_matrix):
    worksheet.write_column(row, col, data)

workbook.close()